In [1]:
import pandas as pd
import numpy as np
import time
import os
import sys
import matplotlib.pyplot as plt
import seaborn as sns


caminho_do_projeto = r'C:\Users\kmenezes\OneDrive - unb.br\tcc_organizacao' # O 'r' antes da string é importante!

if caminho_do_projeto not in sys.path:
    sys.path.insert(0, caminho_do_projeto)

from projeto_aplicado.modelos.otimizador_pli import OtimizadorPLI
from projeto_aplicado.modelos.otimizador_aco import OtimizadorACO
from projeto_aplicado.modelos.otimizador_ag import OtimizadorAG
from projeto_aplicado.modelos.otimizador_base import Otimizador
from projeto_aplicado.modelos.analisador import AnalisadorDeSolucao

# Configurações de visualização
sns.set_theme(style="whitegrid")
pd.set_option('display.max_rows', 100)

In [2]:
CONFIG_BASE = {
    "ARQUIVOS_DADOS": {
        "disciplinas": "disciplinas.csv",
        "professores": "docentes.csv",
        "preferencias": "preferencias.csv",
        "conflitos": "matriz_conflitos_disciplinas.csv" 
    },
    "ALOCACOES_FIXAS": [
        {"professor":"docente_20", "disciplina":"EST0081_01"}, #TCC1  - Leandro
        {"professor":"docente_16", "disciplina":"EST0124_01"}, #LabEst - Joanlise
        {"professor":"docente_21", "disciplina":"EST0088_01"} #Práticas Extensão 2 - Lucas Moreira 
    ]
}

# Parâmetros específicos para cada algoritmo
CONFIG_PLI = {
    **CONFIG_BASE,
    "PENALIDADE_W": 4.0
}

CONFIG_ACO = {
    **CONFIG_BASE,
    "ACO_PARAMS": {
        "n_formigas": 20,
        "n_geracoes": 150,
        "alfa": 1.0,
        "beta": 2.0,
        "taxa_evaporacao": 0.2,
    }
}

CONFIG_AG = {
    **CONFIG_BASE,
    "AG_PARAMS": {
        "n_populacao": 500,
        "n_geracoes": 50,
        "taxa_crossover": 0.85,
        "taxa_mutacao": 0.15,
        "tamanho_torneio": 3,
        "fator_penalidade": 10000
    }
}
analise = AnalisadorDeSolucao(config=CONFIG_BASE)
otimizador = Otimizador(config=CONFIG_BASE)
d1 = otimizador.dados_brutos
prof = d1['professores']
disc = d1['disciplinas']

Dados de contexto carregados pelo analisador.
Analisador de Solução autônomo pronto.
Buscando dados no diretório: C:\Users\kmenezes\OneDrive - unb.br\tcc_organizacao\dados
Dados brutos carregados.
3 alocações fixas validadas e separadas.
28 de 28 professores responderam ao formulário.
Dados preparados. O problema foi reduzido para a otimização.
Otimizador base inicializado. 3 alocações foram fixadas.


C:\Users\kmenezes\OneDrive - unb.br\tcc_organizacao\projeto_aplicado\modelos\otimizador_base.py:82: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_prefs_completo[disciplina_id].fillna(valor_preenchimento, inplace=True)


In [6]:
s252 = pd.read_csv('../dados/solucao_25_1.csv')
rdep = s252.merge(disc[['id_disciplina', 'disciplina']], on='id_disciplina', how='left').merge(prof[['id_docente', 'docente']], on='id_docente',how='left')

In [7]:
analise.avaliar(rdep)

{'escore_total': 124,
 'distribuicao_preferencias': {'pref_3': 40,
  'pref_2': 1,
  'pref_1': 2,
  'pref_0': 4}}

In [8]:
otimizador_pli = OtimizadorPLI(config=CONFIG_PLI)
rpli = otimizador_pli.resolver()['alocacao_final']
rpli = rpli.merge(prof[['id_docente', 'docente']], on ='id_docente',how='left')

Buscando dados no diretório: C:\Users\kmenezes\OneDrive - unb.br\tcc_organizacao\dados
Dados brutos carregados.
3 alocações fixas validadas e separadas.
28 de 28 professores responderam ao formulário.
Dados preparados. O problema foi reduzido para a otimização.
Otimizador base inicializado. 3 alocações foram fixadas.
OtimizadorPLI pronto.

--- Iniciando Processo de Otimização ---
Modelo PLI construído com sucesso.

Iniciando a resolução do modelo PLI...


C:\Users\kmenezes\OneDrive - unb.br\tcc_organizacao\projeto_aplicado\modelos\otimizador_base.py:82: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_prefs_completo[disciplina_id].fillna(valor_preenchimento, inplace=True)


Status da Solução: Optimal
Solução extraída e formatada.
--- Processo de Otimização Concluído ---


In [9]:
analise.avaliar(rpli)

{'escore_total': 132,
 'distribuicao_preferencias': {'pref_3': 43,
  'pref_2': 1,
  'pref_1': 1,
  'pref_0': 2}}

In [10]:
otimizador_aco = OtimizadorACO(config=CONFIG_ACO)
raco = otimizador_aco.resolver()['alocacao_final']
raco = raco.merge(prof[['id_docente', 'docente']], on ='id_docente',how='left')

Buscando dados no diretório: C:\Users\kmenezes\OneDrive - unb.br\tcc_organizacao\dados
Dados brutos carregados.
3 alocações fixas validadas e separadas.
28 de 28 professores responderam ao formulário.
Dados preparados. O problema foi reduzido para a otimização.
Otimizador base inicializado. 3 alocações foram fixadas.
OtimizadorACO pronto.

--- Iniciando Processo de Otimização ---
Matrizes de feromônio e heurística inicializadas.


C:\Users\kmenezes\OneDrive - unb.br\tcc_organizacao\projeto_aplicado\modelos\otimizador_base.py:82: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_prefs_completo[disciplina_id].fillna(valor_preenchimento, inplace=True)


Geração 10/150 | Melhor Qualidade Global: 129
Geração 20/150 | Melhor Qualidade Global: 129
Geração 30/150 | Melhor Qualidade Global: 129
Geração 40/150 | Melhor Qualidade Global: 129
Geração 50/150 | Melhor Qualidade Global: 129
Geração 60/150 | Melhor Qualidade Global: 129
Geração 70/150 | Melhor Qualidade Global: 129
Geração 80/150 | Melhor Qualidade Global: 129
Geração 90/150 | Melhor Qualidade Global: 129
Geração 100/150 | Melhor Qualidade Global: 129
Geração 110/150 | Melhor Qualidade Global: 129
Geração 120/150 | Melhor Qualidade Global: 129
Geração 130/150 | Melhor Qualidade Global: 129
Geração 140/150 | Melhor Qualidade Global: 129
Geração 150/150 | Melhor Qualidade Global: 129
Otimização ACO concluída.
--- Processo de Otimização Concluído ---


In [11]:
analise.avaliar(raco)

{'escore_total': 132,
 'distribuicao_preferencias': {'pref_3': 43,
  'pref_2': 1,
  'pref_1': 1,
  'pref_0': 2}}

In [14]:
otimizador_ag = OtimizadorAG(config=CONFIG_AG)
rag = otimizador_ag.resolver()['alocacao_final']
rag = rag.merge(prof[['id_docente', 'docente']], on ='id_docente',how='left')

Buscando dados no diretório: C:\Users\kmenezes\OneDrive - unb.br\tcc_organizacao\dados
Dados brutos carregados.
3 alocações fixas validadas e separadas.
28 de 28 professores responderam ao formulário.
Dados preparados. O problema foi reduzido para a otimização.
Otimizador base inicializado. 3 alocações foram fixadas.
OtimizadorAG pronto.

--- Iniciando Processo de Otimização ---


C:\Users\kmenezes\OneDrive - unb.br\tcc_organizacao\projeto_aplicado\modelos\otimizador_base.py:82: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_prefs_completo[disciplina_id].fillna(valor_preenchimento, inplace=True)


População inicial com 500 indivíduos criada, respeitando os genes fixos.
Geração 20/50 | Melhor Fitness Global: 124.00
Geração 40/50 | Melhor Fitness Global: 124.00
Otimização AG concluída.
--- Processo de Otimização Concluído ---


In [16]:
analise.avaliar(rag)

{'escore_total': 128,
 'distribuicao_preferencias': {'pref_3': 40,
  'pref_2': 3,
  'pref_1': 2,
  'pref_0': 2}}

In [17]:
disc[['id_disciplina', 'disciplina']].merge(rdep[['id_disciplina','docente', 'preferencia']], on = 'id_disciplina', how='left')\
                                    .merge(rpli[['id_disciplina','docente', 'preferencia']], on = 'id_disciplina', how='left', suffixes=('_dep','_pli'))\
                                    .merge(raco[['id_disciplina', 'docente', 'preferencia']], on = 'id_disciplina', how='left', suffixes=('', '_aco'))\
                                    .merge(rag[['id_disciplina','docente', 'preferencia']], on = 'id_disciplina', how='left', suffixes=('', '_ag')).to_excel('comparacao_resultados.xlsx',index=False)